In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss
data_raw = scipy.io.loadmat('E:\software\EEG+Neuroguide report\EEG_Mat\S024.mat')
f=data_raw['EEG_Data']
eeg=np.array((f))
# detrended fluctuation analysis

def calc_rms(x, scale):
    """
    windowed Root Mean Square (RMS) with linear detrending.
    
    Args:
    -----
      *x* : numpy.array
        one dimensional data vector
      *scale* : int
        length of the window in which RMS will be calculaed
    Returns:
    --------
      *rms* : numpy.array
        RMS data in each window with length len(x)//scale
    """
    # making an array with data divided in windows
    shape = (x.shape[0]//scale, scale)
    X = np.lib.stride_tricks.as_strided(x,shape=shape)
    # vector of x-axis points to regression
    scale_ax = np.arange(scale)
    rms = np.zeros(X.shape[0])
    for e, xcut in enumerate(X):
        coeff = np.polyfit(scale_ax, xcut, 1)
        xfit = np.polyval(coeff, scale_ax)
        # detrending and computing RMS of each window
        rms[e] = np.sqrt(np.mean((xcut-xfit)**2))
    return rms

def dfa(x, scale_lim=[5,9], scale_dens=0.128, show=False):
    """
    Detrended Fluctuation Analysis - measures power law scaling coefficient
    of the given signal *x*.

    More details about the algorithm you can find e.g. here:
    Hardstone, R. et al. Detrended fluctuation analysis: A scale-free 
    view on neuronal oscillations, (2012).

    Args:
    -----
      *x* : numpy.array
        one dimensional data vector
      *scale_lim* = [5,9] : list of length 2 
        boundaries of the scale, where scale means windows among which RMS
        is calculated. Numbers from list are exponents of 2 to the power
        of X, eg. [5,9] is in fact [2**5, 2**9].
        You can think of it that if your signal is sampled with F_s = 128 Hz,
        then the lowest considered scale would be 2**5/128 = 32/128 = 0.25,
        so 250 ms.
      *scale_dens* = 0.25 : float
        density of scale divisions, eg. for 0.25 we get 2**[5, 5.25, 5.5, ... ] 
      *show* = False
        if True it shows matplotlib log-log plot.
    Returns:
    --------
      *scales* : numpy.array
        vector of scales (x axis)
      *fluct* : numpy.array
        fluctuation function values (y axis)
      *alpha* : float
        estimation of DFA exponent
    """
    # cumulative sum of data with substracted offset
    y = np.cumsum(x - np.mean(x))
    scales = (2**np.arange(scale_lim[0], scale_lim[1], scale_dens)).astype(np.int)
    fluct = np.zeros(len(scales))
    # computing RMS for each window
    for e, sc in enumerate(scales):
        fluct[e] = np.sqrt(np.mean(calc_rms(y, sc)**2))
    # fitting a line to rms data
    coeff = np.polyfit(np.log2(scales), np.log2(fluct), 1)
    if show:
        fluctfit = 2**np.polyval(coeff,np.log2(scales))
        plt.loglog(scales, fluct, 'bo')
        plt.loglog(scales, fluctfit, 'r', label=r'$\alpha$ = %0.2f'%coeff[0])
        plt.title('DFA')
        plt.xlabel(r'$\log_{10}$(time window)')
        plt.ylabel(r'$\log_{10}$<F(t)>')
        plt.legend()
        plt.show()
    return scales, fluct, coeff[0]
dfas=np.empty(21)

for i in range (21):
    if __name__=='__main__':
        x = eeg[i]
    # computing DFA of signal envelope
        x = np.abs(ss.hilbert(x))
        scales, fluct, alpha = dfa(x, show=1)
       # print(scales)
        #print(fluct)
        #print("DFA exponent: {}".format(alpha))
        dfas[i]=alpha
        
dfa_fin = np.mean(dfas)
print(round(dfa_fin,4))
dfa_posterior =(dfas[14]+dfas[15]+dfas[18]+dfas[6]+dfas[7]+dfas[8]+dfas[9])/7
print(round(dfa_posterior,4))

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\software\\EEG+Neuroguide report\\EEG_Mat\\S024.mat'

In [2]:
import numpy as np
import scipy.stats

dfa_fcz=np.array([
0.3532,
1.3008,
1.3116,
0.2926,
0.6619,
0.9757,
0.9648,
0.7662,
1.0363,
0.3545,
0.7392,
0.7944,
1.1499,
0.9595,
1.4415,
0.9686,
1.1315,
1.1404,
0.8274,
0.4509,
1.3498,
1.3275,
1.3746,
1.1305])


dfa_mean=np.array([
0.7695,
1.334,
1.3364,
0.411,
0.6951,
0.8539,
1.1981,
1.0123,
1.211,
0.2132,
0.9695,
0.8147,
1.2102,
1.0293,
1.2638,
0.8379,
1.1767,
1.2201,
0.8698,
0.7223,
1.3275,
1.259,
1.1759,
1.2207])

ng_score=np.array([
1,
1,
1,
1,
4.8,
1,
1,
1,
1,
1.3,
1,
4.8,
1,
5,
1,
1,
1,
1,
1,
1,
1,
1,
1.7,
3.8])


hh=scipy.stats.pearsonr(dfa_fcz, ng_score)    # Pearson's r
#scipy.stats.spearmanr(x, y)   # Spearman's rho
#scipy.stats.kendalltau(x, y)  # Kendall's tau
print(hh)


(-0.09441276459594522, 0.6607939074849956)
